In [1]:
from cleanlab_studio import Studio
import pandas as pd
from typing import List, Tuple

In [2]:
# Initialize Cleanlab Studio
studio = Studio("YOUR API KEY HERE")
tlm = studio.TLM(quality_preset="high", options={"model": "gpt-4o", "log": ["explanation"]})

In [3]:
# Load log entries dataset
df = pd.read_csv("application_logs.csv")

In [4]:
def parse_gdpr_violation(response: str) -> bool:
    """
    Extracts the GDPR violation status from the response.
    
    Args:
        response (str): Response from the TLM
    
    Returns:
        bool: True if violates GDPR, False otherwise
    """
    return response.strip().lower() == 'true'

def classify_gdpr_violations(texts: List[str], prompt_template: str) -> Tuple[List[bool], List[float], List[str]]:
    """
    Classifies a list of log entries for GDPR violations using TLM.
    
    Args:
        texts (List[str]): List of log entries to classify
        prompt_template (str): Template string for formatting the prompt
    
    Returns:
        Tuple[List[bool], List[float], List[str]]: A tuple containing three lists:
            - List of GDPR violation statuses for each log entry
            - List of trustworthiness scores for each prediction
            - List of low trustworthiness explanations (empty string if trustworthiness is high)
    """
    prompts = [prompt_template.format(text=text) for text in texts]
    outputs = tlm.try_prompt(prompts)
    
    violations = [parse_gdpr_violation(output['response']) for output in outputs]
    trustworthiness_scores = [output['trustworthiness_score'] for output in outputs]
    low_trust_explanations = [output['log']['explanation'] if output['trustworthiness_score'] < 0.5 else '' for output in outputs]
    
    return violations, trustworthiness_scores, low_trust_explanations

In [5]:
# Define the prompt template
prompt_template = """
Analyze the following application log entry and determine if it violates GDPR by containing or processing personal data without proper safeguards.
Consider things like user identifiers, names, email addresses, IP addresses, or any other personally identifiable information.
Respond with only 'True' if it violates GDPR, or 'False' if it complies with GDPR.

Log entry: {text}

Violates GDPR (True/False):
"""

In [6]:
%%time

# Classify all log entries in batch
violations, trustworthiness_scores, low_trust_explanations = classify_gdpr_violations(df["log_entry"].tolist(), prompt_template)

# Add results to the DataFrame
df['violates_GDPR'] = violations
df['trustworthiness'] = trustworthiness_scores
df['low_trust_explanation'] = low_trust_explanations

Querying TLM... 100%|██████████|

CPU times: user 1min 5s, sys: 17.8 s, total: 1min 23s
Wall time: 2min 9s


In [7]:
# Filter for GDPR violations with high trustworthiness scores
high_trust_threshold = 0.8
gdpr_violations = df[(df['violates_GDPR'] == True) & (df['trustworthiness'] > high_trust_threshold)]

print("Top GDPR-violating log entries (high trustworthiness):")
gdpr_violations[['log_entry', 'violates_GDPR', 'trustworthiness']].head()

Top GDPR-violating log entries (high trustworthiness):


,log_entry,violates_GDPR,trustworthiness
9,2024-09-14 22:23:46 DEBUG: Password reset requ...,True,0.998532
15,2024-09-11 16:27:24 INFO: Geo-location data up...,True,0.995860
22,2024-08-30 10:43:05 DEBUG: User login successf...,True,0.993578
23,2024-08-24 01:31:02 WARN: Newsletter subscript...,True,0.999325
28,2024-08-24 19:32:18 WARN: New user registered:...,True,0.996044


In [8]:
# Filter for examples with low trustworthiness scores
low_trust_threshold = 0.5
low_trust_logs = df[df['trustworthiness'] < low_trust_threshold].sort_values('trustworthiness')

print("\nLog entries with low trustworthiness scores:")
for _, row in low_trust_logs.iterrows():
    print(f"Log entry: {row['log_entry']}")
    print(f"Violates GDPR: {row['violates_GDPR']}")
    print(f"Trustworthiness score: {row['trustworthiness']}")
    print(f"Low trustworthiness explanation: {row['low_trust_explanation']}")
    print("-" * 50)


Log entries with low trustworthiness scores:
Log entry: 2024-09-09 12:25:00 INFO: Unauthorized access attempt to admin panel from IP: 53.125.88.241
Violates GDPR: True
Trustworthiness score: 0.2336285708561434
Low trustworthiness explanation: The answer "True" indicates that the log entry violates GDPR by containing or processing personal data without proper safeguards. However, this assessment is not entirely accurate. The log entry contains an IP address, which can be considered personal data under GDPR if it can be linked to an individual. However, the mere presence of an IP address in a log entry does not automatically constitute a GDPR violation. The context of how the IP address is used, stored, and protected is crucial. If the IP address is used solely for security purposes and is adequately protected, it may not violate GDPR. Therefore, the answer "True" is not definitively correct without additional context.
--------------------------------------------------
Log entry: 2024-0

In [9]:
# Calculate and print summary statistics
total_logs = len(df)
gdpr_violations_count = len(gdpr_violations)
low_trust_count = len(low_trust_logs)

print(f"\nSummary:")
print(f"Total log entries analyzed: {total_logs}")
print(f"GDPR violations (high trustworthiness): {gdpr_violations_count} ({gdpr_violations_count/total_logs:.2%})")
print(f"Low trustworthiness entries: {low_trust_count} ({low_trust_count/total_logs:.2%})")


Summary:
Total log entries analyzed: 1000
GDPR violations (high trustworthiness): 239 (23.90%)
Low trustworthiness entries: 10 (1.00%)


In [10]:
# Save the results
df.to_csv("gdpr_analysis_results.csv", index=False)
print("\nResults saved to 'gdpr_analysis_results.csv'")


Results saved to 'gdpr_analysis_results.csv'
